In [306]:
import numpy as np
import heapq

def parse_map(m):
    return np.array([[int(n) for n in l.strip()] for l in m.splitlines()], dtype=int)

In [445]:
def min_or(a, b, default=0):
    if a is None and b is None:
        return default
    elif a is None:
        return b
    elif b is None:
        return a
    else:
        return min(a, b)
    
def neighbors(v, m):
    i, j = v
    for di, dj in ((1, 0), (0, 1), (-1, 0), (0, -1)):
        if 0 <= i+di < m.shape[0] and 0 <= j+dj < m.shape[1]:
            yield (i+di, j+dj)

def min_path_cost(m):
#     # R-D motion only: (stupid first attempt)
#     m = m.copy()
#     for r in range(m.shape[0]):
#         for c in range(m.shape[1]):
#             cost_l = m[r, c-1] if c > 0 else None
#             cost_t = m[r-1, c] if r > 0 else None
#             m[r, c] += min_or(cost_l, cost_t)
#     return m[-1, -1] - m[0, 0]
    # Motion in any direction:
    q = []
    dist = {}
    source = (0, 0)
    target = (m.shape[0]-1, m.shape[1]-1)
    heapq.heappush(q, (0, source))
    while q:
        du, u = heapq.heappop(q)
        if u == target:
            return du
        for v in neighbors(u, m):
            alt = du + m[v]
            if v not in dist or alt < dist[v]:
                dist[v] = alt
                heapq.heappush(q, (alt, v))
    
def repmap(m, r, c):
    h, w = m.shape
    m = np.tile(m, (r, c))
    for i in range(r):
        for j in range(c):
            m[h*i:h*(i+1), w*j:w*(j+1)] += i + j
    mask = m > 9
    m[mask] = m[mask] % 9
    return m

def debug_route(m):
    d = np.full(m.shape, '.')
    r, c = m.shape
    i, j = r-1, c-1
    while i >= 0 and j >= 0:
        d[i, j] = '#'
        if i == 0 or (j != 0 and m[i, j-1] < m[i-1, j]):
            j -= 1
        else:
            i -= 1
    return '\n'.join(''.join(l) for l in d)

In [446]:
m = parse_map("""1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581""")
m

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [447]:
min_path_cost(m)

40

In [448]:
min_path_cost(repmap(m, 5, 5))

315

In [449]:
%%time
with open('../data/day15.txt') as infile:
    m = parse_map(infile.read())
    print('[p0] Min path cost in 1x1:', min_path_cost(m))
    print('[p1] Min path cost in 5x5:', min_path_cost(repmap(m, 5, 5)))

[p0] Min path cost in 1x1: 447
[p1] Min path cost in 5x5: 2825
CPU times: user 1.26 s, sys: 15.6 ms, total: 1.27 s
Wall time: 1.27 s
